In [1]:
from icecream import ic
import pandas as pd
from dateutil.relativedelta import relativedelta
# from dhs_pca_functions import *
from pandarallel import pandarallel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# Initialization
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# folder: path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
group_by_col = 'cluster number'

urban_rural_all_mode = 'U' # 'all', 'U', 'R'

in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_{urban_rural_all_mode}.pkl"
print(in_f)
# categorical_in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}{add_str}.pkl"
fewsNet_f = f"{input_dir}fewsnet_gaul_adm2.csv"

out_f = f"{input_dir}5_grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc_{urban_rural_all_mode}.pkl"

/mnt/datadisk/data/Projects/water/inputs/grouped_df_V3_HR_cluster number_U.pkl


In [3]:
df = pd.read_pickle(in_f)
df

Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; cluster number                                   
AOGE52FL        1                           Benguela          Angola   
                2                           Benguela          Angola   
                3                           Benguela          Angola   
                4                         Cuanza Sul          Angola   
                5                              Bengo          Angola   
...                                              ...             ...   
ZWGE72FL        390                           Harare        Zimbabwe   
                392                         Bulawayo        Zimbabwe   
                393                           Harare        Zimbabwe   
                396                           Harare        Zimbabwe   
                399                           Harare        Zimbabwe   

                                      Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; cluster number                                       
AOGE52FL        1                           2006           12 2006-12-01   
                2                           2006           11 2006-11-28   
                3                           2006           11 2006-11-22   
                4                           2006           12 2006-12-15   
                5                           2007            1 2007-01-12   
...                                          ...          ...        ...   
ZWGE72FL        390                         2015            7 2015-07-28   
                392                         2015           10 2015-10-28   
                393                         2015           11 2015-11-20   
                396                         2015           10 2015-10-07   
                399                         2015            7 2015-07-07   

                                     Meta; adm2_gaul  \
Meta; GEID_init Meta; cluster number                   
AOGE52FL        1                             Lobito   
                2                             Lobito   
                3                           Benguela   
                4                              Sumbe   
                5                              Dande   
...                                              ...   
ZWGE72FL        390                           Harare   
                392                         Bulawayo   
                393                           Harare   
                396                           Harare   
                399                           Harare   

                                      Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; cluster number                                             
AOGE52FL        1                                                          1.0   
                2                                                          1.0   
                3                                                          1.0   
                4                                                          1.0   
                5                                                          1.0   
...                                                                        ...   
ZWGE72FL        390                                                        0.0   
                392                                                        0.0   
                393                                                        0.0   
                396                                                        0.0   
                399                                                        0.0   

                                      Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; cluster number                                            
AOGE52FL        1                                                         0.0   
                2                                                       

In [4]:
food_security_df = pd.read_csv(fewsNet_f)
food_security_df.head()
ic(food_security_df.columns.to_list())
food_security_df = food_security_df[['adm2_name', 'month', 'year', 'CS', 'HA0']]
food_security_df['HA0'] = food_security_df['HA0'].fillna(0)
food_security_df['IPC + food help'] = food_security_df['CS'] + food_security_df['HA0']
food_security_df.drop(columns=['HA0'], inplace=True)
food_security_df.columns = ['adm2_name', 'month', 'year', 'IPC', 'IPC + food help']
print(len(food_security_df))
food_security_df.sample(10)

/tmp/ipykernel_1715025/2063534321.py:1: DtypeWarning: Columns (26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  food_security_df = pd.read_csv(fewsNet_f)
ic| food_security_df.columns.to_list(): ['OBJECTID',
                                         'adm2_code',
                                         'adm2_name',
                                         'str2_year',
                                         'exp2_year',
                                         'adm1_code',
                                         'adm1_name',
                                         'status',
                                         'disp_area',
                                         'adm0_code',
                                         'adm0_name',
                                         'shape_leng',
                                         'FID_1',
                                         'Shape_Le_1',
                                   

104386


,adm2_name,month,year,IPC,IPC + food help
6067,Nangere,7,2010,3.0,3.0
91388,Lufwanyama,1,2011,1.0,1.0
41608,Boluwaduro,10,2017,1.0,1.0
21406,Kobe-nord-est,7,2013,1.0,2.0
81873,South West Shewa,2,2019,1.0,1.0
85340,Al Qatn,6,2020,2.0,3.0
103904,Chibuto,10,2021,2.0,2.0
21093,Egor,7,2013,1.0,1.0
23658,Akko,1,2014,1.0,1.0
50641,Dire,2,2020,1.0,1.0


In [5]:
def aggregate_ipc(row, food_security_df, start_delta, end_delta, y_m_indicator):
    # Create datetime objects for the start and end dates
    in_date = row['Meta; date']
    if y_m_indicator == 'm':
        start_date = in_date + relativedelta(months=start_delta)
        end_date = in_date + relativedelta(months=end_delta)
    elif y_m_indicator == 'y':
        start_date = in_date + relativedelta(years=start_delta)
        end_date = in_date + relativedelta(years=end_delta)
    else:
        raise ValueError(f"y_m_indicator should be 'm' or 'y', not {y_m_indicator}")
    # Filter the data for the given time span and adm2_gaul
    # print(row.name)
    df_time_span = food_security_df[(food_security_df['date'] >= start_date) & (food_security_df['date'] < end_date) & (food_security_df['adm2_name'] == row.name[1])]


    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC'].mean()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC'].max()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC'].min()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC'].median(), 0)

    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC + food help'].mean()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC + food help'].max()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC + food help'].min()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC + food help'].median(), 0)

    return row

# Convert year and month to datetime
food_security_df['date'] = pd.to_datetime(food_security_df[['year', 'month']].assign(day=1))
print(food_security_df.shape)
print(df.shape)
print('Meta; date' in df.columns)

for start_delta, end_delta, m_y_ind in [(0, 5, 'm'), (6, 12, 'm'), (1, 2, 'y'), (2, 4, 'y'), (4, 6, 'y'), (6, 7, 'y'), (6, 10, 'y'), (10, 15, 'y'), (15, 20, 'y'), (20, 25, 'y'),
                                        (0, 1, 'y'), (0, 2, 'y'), (2, 6, 'y'), (6, 12, 'y'), (12, 20, 'y')]:
    df = df.parallel_apply(aggregate_ipc, axis=1, food_security_df=food_security_df, start_delta=start_delta, 
                                            end_delta=end_delta, y_m_indicator=m_y_ind)

df[df['FS; IPC + FH: 15-20y: mean'].notna()]

(104386, 6)
(24746, 1441)
True


In [ ]:
# save the dataframe
df.to_pickle(out_f)
df.to_csv(out_f.replace('.pkl', '.csv'), index=False)

In [ ]:
df

Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; cluster number                                   
AOGE52FL        1                           Benguela          Angola   
                2                           Benguela          Angola   
                3                           Benguela          Angola   
                4                         Cuanza Sul          Angola   
                5                              Bengo          Angola   
...                                              ...             ...   
ZWGE72FL        390                           Harare        Zimbabwe   
                392                         Bulawayo        Zimbabwe   
                393                           Harare        Zimbabwe   
                396                           Harare        Zimbabwe   
                399                           Harare        Zimbabwe   

                                      Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; cluster number                                       
AOGE52FL        1                           2006           12 2006-12-01   
                2                           2006           11 2006-11-28   
                3                           2006           11 2006-11-22   
                4                           2006           12 2006-12-15   
                5                           2007            1 2007-01-12   
...                                          ...          ...        ...   
ZWGE72FL        390                         2015            7 2015-07-28   
                392                         2015           10 2015-10-28   
                393                         2015           11 2015-11-20   
                396                         2015           10 2015-10-07   
                399                         2015            7 2015-07-07   

                                     Meta; adm2_gaul  \
Meta; GEID_init Meta; cluster number                   
AOGE52FL        1                             Lobito   
                2                             Lobito   
                3                           Benguela   
                4                              Sumbe   
                5                              Dande   
...                                              ...   
ZWGE72FL        390                           Harare   
                392                         Bulawayo   
                393                           Harare   
                396                           Harare   
                399                           Harare   

                                      Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; cluster number                                             
AOGE52FL        1                                                          1.0   
                2                                                          1.0   
                3                                                          1.0   
                4                                                          1.0   
                5                                                          1.0   
...                                                                        ...   
ZWGE72FL        390                                                        0.0   
                392                                                        0.0   
                393                                                        0.0   
                396                                                        0.0   
                399                                                        0.0   

                                      Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; cluster number                                            
AOGE52FL        1                                                         0.0   
                2                                                       